In [3]:
from keras.utils import to_categorical
from keras.applications.inception_v3 import InceptionV3
from keras.applications.densenet import DenseNet121
from keras.preprocessing import image
#from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from keras.preprocessing.image import ImageDataGenerator,  img_to_array, load_img

from keras.layers import Dense, GlobalAveragePooling2D

from keras.applications.xception import Xception, preprocess_input


import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
import subprocess
import os
import pickle
from keras.layers import Input, Conv2D, MaxPooling2D, merge, Lambda,UpSampling2D, concatenate, Reshape, Dropout,Cropping2D,Activation
from keras.models import Model, load_model
import pandas as pd
import sklearn 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.callbacks import Callback
from keras.applications.mobilenet import MobileNet
from keras.applications.nasnet import NASNetMobile,NASNetLarge
import matplotlib.image as mpimg

from xgboost.sklearn import XGBClassifier

import time


In [4]:

from tao_file_preprocess import make_ok_ng_dir 

In [5]:
data_path = './dataset/ARY_ADJ/'
ok_dir,ok_prefix,ng_dir,ng_prefix= make_ok_ng_dir(data_path)

['./dataset/ARY_ADJ/0/NG/AI', './dataset/ARY_ADJ/0/OK/AI', './dataset/ARY_ADJ/1/NG/AI', './dataset/ARY_ADJ/1/OK/AI', './dataset/ARY_ADJ/2/NG/AI', './dataset/ARY_ADJ/3/NG/AI', './dataset/ARY_ADJ/3/OK/AI', './dataset/ARY_ADJ/4/AI', './dataset/ARY_ADJ/5/AI', './dataset/ARY_ADJ/6/OK Natural/AI', './dataset/ARY_ADJ/6/OK Artificial/AI', './dataset/ARY_ADJ/7/NG/AI', './dataset/ARY_ADJ/7/OK/AI', './dataset/ARY_ADJ/8/OK/AI']
./dataset/ARY_ADJ/4/AI 	 311
./dataset/ARY_ADJ/0/OK/AI 	 171
./dataset/ARY_ADJ/1/NG/AI 	 423
./dataset/ARY_ADJ/6/OK Artificial/AI 	 274
./dataset/ARY_ADJ/3/OK/AI 	 301
./dataset/ARY_ADJ/5/AI 	 409
./dataset/ARY_ADJ/0/NG/AI 	 192
./dataset/ARY_ADJ/7/OK/AI 	 418
./dataset/ARY_ADJ/2/NG/AI 	 173
./dataset/ARY_ADJ/6/OK Natural/AI 	 135
./dataset/ARY_ADJ/1/OK/AI 	 404
./dataset/ARY_ADJ/3/NG/AI 	 109
./dataset/ARY_ADJ/7/NG/AI 	 258
./dataset/ARY_ADJ/8/OK/AI 	 253
['./dataset/ARY_ADJ/0/OK/AI', './dataset/ARY_ADJ/1/OK/AI', './dataset/ARY_ADJ/3/OK/AI', './dataset/ARY_ADJ/6/OK Natural

In [6]:
with open('final_dir_files.pkl','rb') as handle:
    
    final_dir_files = pickle.load(handle)

In [7]:
if not os.path.exists('./np_data_model/'): 
    os.makedirs('./np_data_model/')

In [8]:
save_dir = './np_data_model/'

In [12]:

def make_train_data(dirs,finale_dir_files):
    
    final_array =[]

   
    s = time.time()
    for i, dir  in enumerate(dirs):
        img_list=[]
        print ('current dir=', dir)
        
        for im in final_dir_files[dir]:
            s1 =s


            img = load_img(dir+'/'+im, target_size=(224, 224))  # this is a PIL image
            x = img_to_array(img)
            x = preprocess_input(x)
            #print (x.shape)
            #x = cv2.cvtColor(y, cv2.COLOR_BGR2RGB)
            #x = x/127.5 -1.
            img_list.append(x)
            #print ('img_list=', len(img_list))
        testing_data = np.stack(img_list,axis=0)
        #pred = first_prediction(model_1,testing_data)
        final_array.append(testing_data)
        e = time.time()    
        
        print ('batch {}, shape {},time = {}'.format(i,len(final_array),round(e -s1 ,4)))
        s1=e
 
    final_array = np.concatenate(final_array,axis=0)
    print ('train_x_shape',final_array.shape)


    

    print ('train_x_saved')

    e = time.time()
    print ('time=', round(e -s ,4))
    
    return final_array


In [13]:
ok_train= make_train_data(ok_dir,final_dir_files)
np.save(save_dir +'train_x_ok_just_image'+'.npy',ok_train)
ok_label = np.zeros((ok_train.shape[0],1))
print ('ok image shape=', ok_train.shape, 'ok_label = ', ok_label.shape, 'ok image data completed')

current dir= ./dataset/ARY_ADJ/0/OK/AI
batch 0, shape 1,time = 2.7213
current dir= ./dataset/ARY_ADJ/1/OK/AI
batch 1, shape 2,time = 9.1113
current dir= ./dataset/ARY_ADJ/3/OK/AI
batch 2, shape 3,time = 13.936
current dir= ./dataset/ARY_ADJ/6/OK Natural/AI
batch 3, shape 4,time = 16.066
current dir= ./dataset/ARY_ADJ/6/OK Artificial/AI
batch 4, shape 5,time = 19.8781
current dir= ./dataset/ARY_ADJ/7/OK/AI
batch 5, shape 6,time = 26.6048
current dir= ./dataset/ARY_ADJ/8/OK/AI
batch 6, shape 7,time = 30.6273
train_x_shape (1950, 224, 224, 3)
train_x_saved
time= 30.7792
ok image shape= (1950, 224, 224, 3) ok_label =  (1950, 1) ok image data completed


In [14]:
ng_train= make_train_data(ng_dir,final_dir_files)
np.save(save_dir +'train_x_ng_just_image'+'.npy',ng_train)
ng_label = np.ones((ng_train.shape[0],1))
print ('ng image shape=', ng_train.shape, 'ng_label = ', ng_label.shape, 'ng image data completed')

current dir= ./dataset/ARY_ADJ/0/NG/AI
batch 0, shape 1,time = 3.0459
current dir= ./dataset/ARY_ADJ/1/NG/AI
batch 1, shape 2,time = 9.7276
current dir= ./dataset/ARY_ADJ/2/NG/AI
batch 2, shape 3,time = 12.5216
current dir= ./dataset/ARY_ADJ/3/NG/AI
batch 3, shape 4,time = 14.2625
current dir= ./dataset/ARY_ADJ/4/AI
batch 4, shape 5,time = 19.2918
current dir= ./dataset/ARY_ADJ/5/AI
batch 5, shape 6,time = 25.8771
current dir= ./dataset/ARY_ADJ/7/NG/AI
batch 6, shape 7,time = 30.0645
train_x_shape (1869, 224, 224, 3)
train_x_saved
time= 30.2116
ng image shape= (1869, 224, 224, 3) ng_label =  (1869, 1) ng image data completed


In [17]:
final_train_x = np.concatenate([ok_train,ng_train],axis=0)
np.save(save_dir+'final_train_x_just_image.npy',final_train_x)
print ('final_train_x shape', final_train_x.shape, 'saved')
final_train_y = np.concatenate([ok_label,ng_label],axis=0)

np.save(save_dir+'final_train_y_just_image.npy',final_train_y)
print ('final_train_y shape', final_train_y.shape, 'saved')

final_train_x shape (3819, 224, 224, 3) saved
final_train_y shape (3819, 1) saved


In [16]:
import datetime
print(datetime.datetime.now())

2018-11-14 00:15:20.083233
